In [62]:
import numpy as np
import neurolab as nl
import random
from PIL import Image
import os
import re

In [40]:
def mat2vec(x):
    m = x.shape[0]*x.shape[1]
    tmp1 = np.zeros(m)

    c = 0
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            tmp1[c] = x[i,j]
            c +=1
    return tmp1

In [41]:
def readImg2array(file,size, threshold= 145):
    pilIN = Image.open(file).convert(mode="L")
    pilIN= pilIN.resize(size)
    #pilIN.thumbnail(size,Image.ANTIALIAS)
    imgArray = np.asarray(pilIN,dtype=np.uint8)
    x = np.zeros(imgArray.shape,dtype=float)
    x[imgArray > threshold] = 1
    x[x==0] = -1
    return x

In [42]:
def array2img(data, outFile = None):
    #data is 1 or -1 matrix
    y = np.zeros(data.shape,dtype=np.uint8)
    y[data==1] = 255
    y[data==-1] = 0
    img = Image.fromarray(y,mode="L")
    if outFile is not None:
        img.save(outFile)
    return img

In [52]:
current_path = os.getcwd()
path = current_path+"/target_pics/"
target_paths = []
names = []

for i in os.listdir(path):
    if re.match(r'..jpg',i):
        target_paths.append(path+i)

for path in target_paths:
    name = path.split("/")[-1].split(".")[0]
    names.append(name)

target_paths


['c:\\Users\\nicco\\Desktop\\lab-4-I.A-vscode/target_pics/A.jpg',
 'c:\\Users\\nicco\\Desktop\\lab-4-I.A-vscode/target_pics/B.jpg',
 'c:\\Users\\nicco\\Desktop\\lab-4-I.A-vscode/target_pics/I.jpg',
 'c:\\Users\\nicco\\Desktop\\lab-4-I.A-vscode/target_pics/T.jpg',
 'c:\\Users\\nicco\\Desktop\\lab-4-I.A-vscode/target_pics/Y.jpg']

In [63]:
size = (100,100)
threshold = 60
target = []

for path in target_paths:
    x = readImg2array(path,size,threshold)
    x = mat2vec(x)
    target.append(x)

target = np.array(target)

In [64]:
net = nl.net.newhop(target)
#,transf= nl.trans.HardLims()

In [55]:
noise_percent = 0.1
test = []
rango = range(0,size[0]*size[1])
noise_level = round((rango[-1]+1)*noise_percent)

for i in range (len(target)):
    noise = random.sample(rango, noise_level)
    image_noisy = target[i].copy()
    image_noisy[noise] = -image_noisy[noise]
    test.append(image_noisy)
    array2img(image_noisy.reshape(size),outFile = target_paths[i].replace("target_pics","test_pics"))

test = np.array(test)

In [60]:
output_target = net.sim(target)

for i in range(len(output_target)):
    print(names[i], ": ", (output_target[i] == target[i]).all(), 'Sim. steps', len(net.layers[0].outs))
    array2img(output_target[i].reshape(size),outFile = "./output/"+names[i]+"_target.jpg")


A :  [ True  True  True ...  True  True  True] Sim. steps 2
B :  [ True  True  True ...  True  True  True] Sim. steps 2
I :  [ True  True  True ...  True  True  True] Sim. steps 2
T :  [ True  True  True ...  True  True  True] Sim. steps 2
Y :  [ True  True  True ...  True  True  True] Sim. steps 2


In [59]:
output_test = net.sim(test)

for i in range(len(output_target)):
    print(names[i],": ",(output_test[i] == target[i]).all(), 'Sim. steps', len(net.layers[0].outs))
    array2img(output_test[i].reshape(size),outFile = "./output/"+names[i]+"_test.jpg")


A :  False Sim. steps 2
B :  False Sim. steps 2
I :  False Sim. steps 2
T :  False Sim. steps 2
Y :  False Sim. steps 2
